In [1]:
import pandas as pd

In [5]:
df = pd.read_csv("~/Downloads/account.csv")
df = df.drop("Account No", axis=1)
df = df.map(lambda value: value.replace("1_", "") if isinstance(value, str) else None)
df

,Parent Key Id,Key Id
0,None,$ROOT
1,0,$UNOWNED
2,CASH,11000
3,CASH,11100
4,CASH,11200
...,...,...
278,OPINC,TOTOPEXP
279,OPINC,TOTOPREV
280,$UNOWNED,TOTRSTC&I
281,$UNOWNED,TOTUNRCA


In [6]:
from collections import defaultdict
import json

# Simulated data from your image
rows = df.to_dict("records")

# Step 1: Build a tree structure (parent -> [children])
children = defaultdict(list)
for row in rows:
    parent = row["Parent Key Id"]
    child = row["Key Id"]
    children[parent].append(child)

# Step 2: Recursive function to build nested dict
def build_tree(node):
    if node not in children:
        return node  # leaf node
    child_list = children[node]
    result = []
    for child in child_list:
        if child in children:
            result.append({child: build_tree(child)})
        else:
            result.append(child)
    return result

# Build from root (None)
tree = {children[None][0]: build_tree(children[None][0])}

# Pretty print the result
print(json.dumps(tree, indent=2))


{
  "$ROOT": [
    {
      "ALL": [
        {
          "NETASSETS": [
            "30100",
            "30101",
            "30200",
            "30300",
            "30400",
            "30500",
            {
              "UNA": [
                "30000",
                {
                  "OPINC": [
                    {
                      "TOTOPEXP": [
                        {
                          "DEPAMI": [
                            "80000",
                            "81000",
                            "82000",
                            "83000"
                          ]
                        },
                        {
                          "EXP-NOD&A": [
                            {
                              "CERTOBLGN": [
                                "48000",
                                "48100",
                                "48900",
                                "79050"
                              ]
                            },
  

In [9]:
import os

file_path = os.path.expanduser("~/Downloads/hierarchy.json")

with open(file_path, "w") as f:
    json.dump(tree, f, indent=2)